<a href="https://colab.research.google.com/github/llNeeleshll/bert_sentence_clustring/blob/main/Bert_Sentence_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install kmeans-pytorch

In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [ ]:
class BertEmbeddings:

  def __init__(self):
    self.model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    
    
  def embed_text(self, text_to_embed):
    
    marked_text = "[CLS] " + text_to_embed + " [SEP]"
    tokenized_text = self.tokenizer.tokenize(marked_text)
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
      
      outputs = self.model(tokens_tensor, segments_tensors)
      # Evaluating the model will return a different number of objects based on 
      # how it's  configured in the `from_pretrained` call earlier. In this case, 
      # becase we set `output_hidden_states = True`, the third item will be the 
      # hidden states from all layers. See the documentation for more details:
      # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
      hidden_states = outputs[2]
      
    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)
    
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)

    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)

    # Stores the token vectors, with shape [31 x 768]
    token_vecs_sum = []

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec = torch.sum(token[-4:], dim=0)
        
        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)

    total_vector_sum = torch.zeros([1,768])

    for i in range(len(token_vecs_sum)):
        total_vector_sum += token_vecs_sum[i]

    return total_vector_sum


In [84]:
be = BertEmbeddings()
import numpy  as np
x = ["I hate it",
     "I really love to play cricket",
     "This is very bad. You should kill him", 
     "This is awesome, I am looking to work"]

#encoded_x = be.embed_text(x[1])
l = [be.embed_text(x[i]) for i in range(0, len(x))]

#print(encoded_x.shape)

l = tuple(l)
encoded_x = torch.stack((l), dim=0)
print(encoded_x.data)
print(encoded_x.shape)

#et = be.embed_text("deposite money in a bank")
#et2 = be.embed_text("This is the best")

tensor([[[  1.9738,  10.4972,  -5.3046,  ...,   5.5136,   3.6367,   6.2034]],

        [[ 23.1837,   2.3267,  -2.1827,  ...,  -0.0416,  10.5470, -11.5288]],

        [[  7.6840,   3.7987,   1.6742,  ...,  -0.6801,   5.2528,   1.1538]],

        [[ 11.2275,   5.4036,  14.8448,  ..., -13.0721,   1.6352,   1.5287]]])
torch.Size([4, 1, 768])


In [ ]:
from kmeans_pytorch import kmeans

cluster_ids_x, cluster_centers = kmeans(
    X=encoded_x, num_clusters=2, distance='euclidean')

print(cluster_ids_x)

[running kmeans]: 2it [00:00, 159.52it/s, center_shift=0.000000, iteration=2, tol=0.000100]

running k-means on cpu..
tensor([1, 1, 1, 0])


In [ ]:
from scipy.spatial.distance import cosine

# Calculate the cosine similarity between the sentences 

sim = (1 - cosine(et, et2)) * 100
from kmeans_pytorch import kmeans
print("The similarity is : %.2f" % sim, "%")

In [ ]:
from kmeans_pytorch import kmeans_predict

ex = "i dont like food"
ex = be.embed_text(ex)
print(ex.shape)
cluster_ids_y = kmeans_predict(
    ex , cluster_centers, 'euclidean'
)

torch.Size([1, 768])
predicting on cpu..


IndexError: ignored

In [ ]:
plt.figure(figsize=(4, 3), dpi=160)
plt.scatter(x[:, 0], x[:, 1], c=cluster_ids_x, cmap='cool')
plt.scatter(y[:, 0], y[:, 1], c=cluster_ids_y, cmap='cool', marker='X')
plt.scatter(
    cluster_centers[:, 0], cluster_centers[:, 1],
    c='white',
    alpha=0.6,
    edgecolors='black',
    linewidths=2
)
plt.axis([-1, 1, -1, 1])
plt.tight_layout()
plt.show()


In [ ]:
s1 = torch.empty(0,3)
s2 = torch.tensor([4,5,6])
s2 = torch.stack((s1,s2), dim=0)

print(s2)

RuntimeError: ignored

In [ ]:
be = BertEmbeddings()
import numpy  as np
x1 = ["I hate it",
     "I really love to play cricket",
     "This is very bad. You should kill him", 
     "This is awesome, I am looking to work"]

#encoded_x = be.embed_text(x[1])
l1 = [be.embed_text(x1[i]) for i in range(0, len(x1))]

numy = np.array(l1)
#print(encoded_x.shape)

l1 = tuple(l1)
encoded_x = torch.stack((l1), dim=0)
print(encoded_x.data)
print(encoded_x.shape)

#et = be.embed_text("deposite money in a bank")
#et2 = be.embed_text("This is the best")

ValueError: ignored

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(numy)


NameError: ignored